# Runkeeper munging

https://runkeeper.com/exportData

* make an export
* download the ZIP
* unzip it into `./export/`

In [1]:
from lxml.etree import parse,dump
from pathlib import Path
from osgeo import ogr
import datetime
import geopandas as gpd
import folium
from shapely.geometry import Polygon, LineString
import json

p = Path('export')
walks = list(p.glob('*.gpx'))

In [2]:
def parse_gpx(p):
    doc = parse(p.open())
    name = doc.xpath('gpx:trk/gpx:name',namespaces={'gpx': 'http://www.topografix.com/GPX/1/1'})[0].text
    time = doc.xpath('gpx:trk/gpx:time',namespaces={'gpx': 'http://www.topografix.com/GPX/1/1'})[0].text
    route = [(float(trkpt.attrib['lon']),float(trkpt.attrib['lat'])) 
             for trkpt in doc.xpath('//gpx:trkpt',namespaces={'gpx': 'http://www.topografix.com/GPX/1/1'})]
    d = {
        'name': name,
        'time': datetime.datetime.strptime(time, "%Y-%m-%dT%H:%M:%SZ"),
        'geometry': LineString(route)
    }
    
    return d


In [3]:
%matplotlib inline
routes = [parse_gpx(walk) for walk in walks]

In [4]:
df = gpd.GeoDataFrame(routes)
df.sort_values('time',inplace=True)


In [13]:
def style_function(feature):
    return {
        'weight': 2,
        'opacity': .5,
        'color': 'red' if feature['properties']['type'] == 'Cycling' else 'green'
    }

home = [42.045465,-87.7161625]
m = folium.Map(
    location=[42.03,-87.7161625],#home,
    tiles='OpenStreetMap',
    zoom_start=15,
    height=2500
)

df['type'] = df['name'].apply(lambda x: x.split(' ')[0])
j = df.sort_values('time').drop('time',axis=1).to_json()
# j = json.loads(df['geometry'].to_json())
# for i,feature in enumerate(j['features']):
#     row = df.loc[i].to_dict()
#     feature['properties']['name'] = row['name']
#     feature['properties']['type'] = row['name'].split(' ')[0]
#     feature['properties']['date'] = row['time'].strftime("%Y-%m-%d"),

folium.GeoJson(
    j,
    name='geojson',
    style_function=style_function
).add_to(m)

m

In [6]:
df

,name,time,geometry
45,Walking 3/13/20 5:29 pm,2020-03-13 22:29:47,"LINESTRING (-87.715621 42.050955, -87.715615 4..."
75,Walking 3/14/20 11:48 am,2020-03-14 16:48:29,"LINESTRING (-87.71584199999999 42.050468, -87...."
35,Walking 3/14/20 6:12 pm,2020-03-14 23:12:51,"LINESTRING (-87.715424 42.050434, -87.715487 4..."
10,Walking 3/15/20 10:37 am,2020-03-15 15:37:14,"LINESTRING (-87.71023 42.038141, -87.710254000..."
64,Walking 3/15/20 4:59 pm,2020-03-15 21:59:23,"LINESTRING (-87.71553299999999 42.050437, -87...."
...,...,...,...
36,Walking 5/19/20 5:18 pm,2020-05-19 22:18:15,"LINESTRING (-87.71557799999999 42.050591, -87...."
68,Cycling 5/20/20 8:20 am,2020-05-20 13:20:51,"LINESTRING (-87.715497 42.050704, -87.715493 4..."
48,Cycling 5/20/20 8:56 am,2020-05-20 13:56:05,"LINESTRING (-87.71566 42.050477, -87.715641000..."
19,Walking 5/21/20 8:26 am,2020-05-21 13:26:31,"LINESTRING (-87.71560599999999 42.05049, -87.7..."


'{"type": "FeatureCollection", "features": [{"id": "45", "type": "Feature", "properties": {"name": "Walking 3/13/20 5:29 pm", "type": "Walking"}, "geometry": {"type": "LineString", "coordinates": [[-87.715621, 42.050955], [-87.715615, 42.050997], [-87.715609, 42.05109], [-87.715616, 42.05119], [-87.715613, 42.051285], [-87.715602, 42.051379], [-87.715611, 42.05147], [-87.715603, 42.05157], [-87.715601, 42.05166], [-87.715601, 42.051762], [-87.715619, 42.051853], [-87.715756, 42.051852], [-87.715877, 42.051843], [-87.716005, 42.051815], [-87.716135, 42.051796], [-87.71627, 42.051797], [-87.716399, 42.051799], [-87.716523, 42.051789], [-87.716658, 42.051786], [-87.716787, 42.051781], [-87.716909, 42.051778], [-87.717031, 42.051774], [-87.717161, 42.051779], [-87.717299, 42.051776], [-87.717435, 42.05176], [-87.717564, 42.051768], [-87.717697, 42.051773], [-87.717819, 42.051791], [-87.717925, 42.051861], [-87.717969, 42.051947], [-87.717957, 42.052048], [-87.717939, 42.052147], [-87.71794